In [ ]:
from __future__ import print_function

In [ ]:
from nbtemplate import display_header, get_path
display_header('SelectRowlandParameters.ipynb', status='in development')

In [ ]:
import os
import sys
from glob import glob
import functools

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table
import astropy.units as u

%matplotlib inline

In [ ]:
filelist = glob(os.path.join(get_path('grid2designtorus'), '*.fits'))

In [ ]:
filelist.sort()  # for display purposes. Also, makes the order more predictable
len(filelist)

In [ ]:
p = Table.read(filelist[-4])
p['wave'] = p['energy'].to(u.Angstrom, equivalencies=u.spectral())

In [ ]:
p['probability'].sum()

In [ ]:
zero_pos(p)

In [ ]:
p = p.group_by('wave')

In [ ]:
p_prob = p['wave', 'probability'].groups.aggregate(np.mean)

In [ ]:
# Factor 4 is there because we simulate only one channel in this grid for speed.
plt.plot(p_prob['wave'], p_prob['probability'] * p_prob.meta['A_GEOM'] * 4)

# This sill incluces everything - Photons that miss the detector, 0th order and dispersed etc.

In [ ]:
def zero_pos(p, coord='circ_phi'):
    ind = (p['order'] == 0) & np.isfinite(p[coord])
    return np.mean(p[coord][ind])

def angle_covered_by_CCDs(p, n_ccds=8):
    return 8 * 25. / (2. * np.pi * p.meta['CIRCLE_R'])

In [ ]:
# calculate which angles can be covered
phi_0 = zero_pos(p)
phi_ccd = angle_covered_by_CCDs(p)

# phi at the point excatly half-way between the two channels, the "middle"
phi_m = phi_0 - np.arcsin(p.meta['D_CHAN'] / 2 / p.meta['CIRCLE_R'])

# make new column "distance from phi_m"
p['phi_folded'] = np.abs(p['circ_phi'] - phi_m)

In [ ]:
p.meta

In [ ]:
angle_covered_by_CCDs(p)

In [ ]:
ind = np.isfinite(p['circ_phi'])
out = plt.hist(p['phi_folded'][ind], weight, bins=100, range=[0,.1])

In [ ]:
indgood = np.isfinite(p['order']) & np.isfinite(p['circ_phi'])
inddisp = indgood & (p['order'] < 0)
ind0 = indgood & (p['order'] == 0)

In [ ]:
indwave = p['wave'] > 12.
hist, bin_edges = np.histogram(p['circ_phi'][indwave], weights=p['probability'][indwave])

In [ ]:
p_prob = p['wave', 'probability'].groups.aggregate(np.mean)
# Factor 4 is there because we simulate only one channel in this grid for speed.
#plt.plot(p_prob['wave'], p_prob['probability'] * p_prob.meta['A_GEOM'] * 4)
phistart = 0
indang = (p['circ_phi'] > phistart) & (p['circ_phi'] < (phistart + angle_covered_by_CCDs(p) + 1.)) & inddisp
p_t = p[indang]
p_prob = p_t.group_by('wave')
p_prob = p_prob['wave', 'probability'].groups.aggregate(np.mean)
plt.plot(p_prob['wave'], p_prob['probability'] * p.meta['A_GEOM'] * 4)


In [ ]:
p.colnames

In [ ]:
ind = np.isfinite(p['circ_phi'])
out = plt.hist(p['circ_phi'][ind], bins=100)